In [59]:
!pip install sqlalchemy
!pip install pyodbc


In [60]:
from sqlalchemy import create_engine, MetaData, Table, inspect
import pandas as pd
import logging
import pytest

In [92]:
# configure logging
logging.basicConfig(filename='py_log2.log', level=logging.ERROR,
                   format='%(asctime)s - %(levelname)s - %(message)s')

# create engine function
def new_engine(dialect, server, database, user=None, password=None, integrated_security=True):
    try:
        if integrated_security:
            # For Windows authentication:
            eng = f"{dialect}://{server}/{database}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server"
        else:
            eng = f"{dialect}://{user}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server"
        print(f'Connecting to: {eng}.')
        return create_engine(eng)
    except Exception as e:
        logging.error(f'Database engine creation failed: {e}.')
        raise # Re-raise exception


In [94]:
# connect to SQL Server
try:
    engine = new_engine('mssql', 'MARINA', 'ExampleDatabase', integrated_security=True)
    connection = engine.connect()
    print('Connection to database successfull.')
except Exception as e:
    logging.error(f'Connection to database failed: {e}')
    raise # stop if connection fails

Connecting to: mssql://MARINA/ExampleDatabase?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server.
Connection to database successfull.


In [96]:
# read data from SQL Table
TABLE_NAME = "dbo.Customer"

try:
    query = f'SELECT * FROM {TABLE_NAME}'
    df = pd.read_sql(query, connection)
    print(df.head())
except Exception as e:
    logging.error(f'Failed to read data: {e}')
    raise


            SSNO FirstName    LastName       City
0  19560113-1313     Kalle    Karlsson  Stockholm
1  19920415-4534       Kim    Alfström       None
2  20030603-7834     Julia   Johansson      Malmö
3  20080107-3341     Sofia  Edvardsson   Göteborg


In [98]:
# process and update data

try:
    df["LastName"] = df["LastName"].str.upper()
except Exception as e:
    logging.error(f'Data processing failed: {e}')
    raise   

try:
    df.to_sql(TABLE_NAME, con=engine, if_exists="replace", index=False)
    print("SQL table is updated!")
except Exception as e:
    logging.error(f'Failed to update SQL table: {e}')
    raise

connection.close()


SQL table is updated!


In [102]:
# running the test script

pytest.main(["-q", "test_script.py"])

...                                                                      [100%]
3 passed in 0.11s


<ExitCode.OK: 0>